In [1]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import loss_RA_MIL
from transformers import get_linear_schedule_with_warmup
from model import Model
from model_FC import ModelFC

import numpy as np
import torch
import matplotlib.pyplot as plt

import itertools
import torch
import einops
import torch.nn.functional as F

from transformers import LxmertModel, LxmertTokenizer
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from loss import loss_RA_MIL

from model import *

PyTorch version 1.6.0 available.


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
ACTION = '[unused3]'

lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
lxmert_tokenizer.add_special_tokens({"additional_special_tokens": [ACTION]})
lxmert_tokenizer.encode([ACTION])

lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
lxmert.to(device)

ACTION_TOKEN = lxmert_tokenizer.convert_tokens_to_ids(ACTION)

In [4]:
num_actions = 8
MAX_DETECTIONS=20
batch_size = 1

DETECTION_EMBEDDING_SIZE = 2048
OUTPUT_EMBEDDING_SIZE = 768
NUM_FRAMES_PER_STEP=5
MAX_DETECTIONS=20
CANDIDATES = NUM_FRAMES_PER_STEP * MAX_DETECTIONS

dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)

In [5]:
model = Model(device, MAX_DETECTIONS=20)

In [6]:
_, boxes, features, steps_list, entity_list, entity_count_list, _, _ = next(iter(dataloader))

In [7]:
steps_list = remove_unused2(steps_list)

In [8]:
entities_count = entity_count_list[0]
entities = entity_list[0]
steps = steps_list[0]

In [9]:
steps = [step.strip() for step in steps.split(ACTION)[:-2]]

In [10]:
boxes = boxes.squeeze(0)
boxes = boxes.reshape(num_actions, CANDIDATES, -1)
boxes = boxes.to(device)

features = features.squeeze(0)
features = features.reshape(num_actions, CANDIDATES, -1)
features = features.to(device)

In [11]:
boxes.shape

torch.Size([8, 100, 4])

In [12]:
from random import choice
from random import seed

seed(0)

steps_pairs = []
bboxes_pairs = torch.zeros(num_actions, 2, CANDIDATES, 4)
features_pairs = torch.zeros(num_actions, 2, CANDIDATES, DETECTION_EMBEDDING_SIZE)
entity_list_pairs = []
num_steps = len(steps)
pairs = []

for idx, step in enumerate(steps):
    idx_1 = idx
    idx_2 = choice([idx_2 for idx_2 in range(num_steps) if idx_2 != idx_1])
    
    pairs.append((idx_1, idx_2))
    
    steps_pairs.append(steps[idx_1] + " " + ACTION + " " + steps[idx_2] + " " + ACTION + " " + ACTION)
    
    bboxes_pairs[idx_1][0] = boxes[idx_1]
    bboxes_pairs[idx_1][1] = boxes[idx_2]
    
    features_pairs[idx_1][0] = features[idx_1]
    features_pairs[idx_1][1] = features[idx_2]
    
    entity_list_pairs.append([entities[idx_1], entities[idx_2]])
    
bboxes_pairs = bboxes_pairs.reshape(num_actions, 2 * CANDIDATES, -1)
bboxes_pairs = bboxes_pairs.to(device)

features_pairs = features_pairs.reshape(num_actions, 2 * CANDIDATES, DETECTION_EMBEDDING_SIZE)
features_pairs = features_pairs.to(device)

entity_count = [len(action) for action in sum(entity_list_pairs, [])]

In [13]:
entity_list_pairs


[[['garram masala seeds', 'the oil'], ['chili', 'the pot']],
 [['the lamb', 'the pot'], ['water', 'the pot']],
 [['garlic ginger paste'], ['chili', 'the pot']],
 [['chili tumeric coriander cumin'], ['water', 'the pot']],
 [['water', 'the pot'], ['garram masala seeds', 'the oil']],
 [['potatos', 'the pot'], ['garlic ginger paste']],
 [['the tomatos', 'the pot'], ['water', 'the pot']],
 [['chili', 'the pot'], ['chili tumeric coriander cumin']]]

In [14]:
#2*num_actions - entities in action 1 (pair 1), action 2 (pair 1), action 1 (pair 2) action 2 (pair 2) etc.
entity_count

[2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1]

In [15]:
boxes.shape

torch.Size([8, 100, 4])

In [16]:
pairs

[(0, 7), (1, 4), (2, 7), (3, 4), (4, 0), (5, 2), (6, 4), (7, 3)]

In [17]:
steps_pairs

['add garram masala seeds and a bay leaf to the oil. [unused3] add chili to the pot. [unused3] [unused3]',
 'add the lamb to the pot. [unused3] add water to the pot. [unused3] [unused3]',
 'add garlic ginger paste and chopped onions to the pot. [unused3] add chili to the pot. [unused3] [unused3]',
 'add chili tumeric coriander cumin and salt. [unused3] add water to the pot. [unused3] [unused3]',
 'add water to the pot. [unused3] add garram masala seeds and a bay leaf to the oil. [unused3] [unused3]',
 'add potatos to the pot. [unused3] add garlic ginger paste and chopped onions to the pot. [unused3] [unused3]',
 'add the tomatos to the pot. [unused3] add water to the pot. [unused3] [unused3]',
 'add chili to the pot. [unused3] add chili tumeric coriander cumin and salt. [unused3] [unused3]']

In [18]:
entity_list_pairs

[[['garram masala seeds', 'the oil'], ['chili', 'the pot']],
 [['the lamb', 'the pot'], ['water', 'the pot']],
 [['garlic ginger paste'], ['chili', 'the pot']],
 [['chili tumeric coriander cumin'], ['water', 'the pot']],
 [['water', 'the pot'], ['garram masala seeds', 'the oil']],
 [['potatos', 'the pot'], ['garlic ginger paste']],
 [['the tomatos', 'the pot'], ['water', 'the pot']],
 [['chili', 'the pot'], ['chili tumeric coriander cumin']]]

In [19]:
print(bboxes_pairs.shape)
print(features_pairs.shape)

torch.Size([8, 200, 4])
torch.Size([8, 200, 2048])


In [18]:
inputs = lxmert_tokenizer(
            steps_pairs,
            padding="longest",
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

inputs.input_ids = inputs.input_ids.to(device)
inputs.attention_mask = inputs.attention_mask.to(device)
inputs.token_type_ids = inputs.token_type_ids.to(device)

output = lxmert(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            visual_feats=features_pairs,
            visual_pos=bboxes_pairs,
            token_type_ids=inputs.token_type_ids,
            return_dict=True,
            output_attentions=True
        )

In [62]:
entity_idx

[[[2, 7], [13, 14], [18, 18], [20, 21]],
 [[2, 3], [5, 6], [10, 10], [12, 13]],
 [[2, 4], [14, 14], [16, 17]],
 [[2, 9], [15, 15], [17, 18]],
 [[2, 2], [4, 5], [9, 14], [20, 21]],
 [[2, 3], [5, 6], [10, 12]],
 [[2, 4], [6, 7], [11, 11], [13, 14]],
 [[2, 2], [4, 5], [9, 16]]]

In [64]:
action_idx

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True, False, False, False,
         False, False, False,  True,  True, False],
        [False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False,  True,  True, False, False, False,
         False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False,  True,
          True, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
          True,  True, False, False, False, False],
        [False, False, False, False, False, False, False,  True, False, False,
         False, False, False, False, False, False, False, False, False, False,
  

In [63]:
entity_list_pairs

[[['garram masala seeds', 'the oil'], ['chili', 'the pot']],
 [['the lamb', 'the pot'], ['water', 'the pot']],
 [['garlic ginger paste'], ['chili', 'the pot']],
 [['chili tumeric coriander cumin'], ['water', 'the pot']],
 [['water', 'the pot'], ['garram masala seeds', 'the oil']],
 [['potatos', 'the pot'], ['garlic ginger paste']],
 [['the tomatos', 'the pot'], ['water', 'the pot']],
 [['chili', 'the pot'], ['chili tumeric coriander cumin']]]

In [85]:
entity_idx

[[[2, 7], [13, 14], [18, 18], [20, 21]],
 [[2, 3], [5, 6], [10, 10], [12, 13]],
 [[2, 4], [14, 14], [16, 17]],
 [[2, 9], [15, 15], [17, 18]],
 [[2, 2], [4, 5], [9, 14], [20, 21]],
 [[2, 3], [5, 6], [10, 12]],
 [[2, 4], [6, 7], [11, 11], [13, 14]],
 [[2, 2], [4, 5], [9, 16]]]

In [86]:
entity_count

[2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1]

In [120]:
entity_idx = get_ent_inds(model, entity_list_pairs, steps_pairs)
entity_embeddings = get_entity_embeddings(output['language_output'], entity_idx).split(entity_count)

action_idx = (inputs.input_ids == ACTION_TOKEN)
A = output['language_output'][action_idx]

V = output['vision_output']

IndexError: The shape of the mask [8, 28] at index 1 does not match the shape of the indexed tensor [8, 26, 768] at index 1

In [74]:
entity_count

[2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1]

In [75]:
#entity_embeddings is 16 (tuple) x entity_count x 768
entity_embeddings[4].size()

torch.Size([1, 768])

In [84]:
len(entity_embeddings)

16

In [20]:
#gather entity embeddings in each pair together
E = []

for i in range(num_actions):
    idx_1 = 2 * i
    idx_2 = idx_1 + 1
    E.append([entity_embeddings[idx_1], entity_embeddings[idx_2]])

In [94]:
#embedding of entity 1 in action 1 of pair 1
len(E[0][0][1])

768

In [101]:
#entity embeddings in action step i, pair 1
E[2][0].size()

torch.Size([1, 768])

In [100]:
#the visual embeddings in action step i (200 - for both pairs)
V[0].size()

torch.Size([200, 768])

In [118]:
(E[0][0][0]*V[0][:100]).sum(dim=-1)

tensor([-42.8657, -35.3234, -45.7928, -30.0132, -21.6943, -16.2853, -17.9074,
        -29.7822, -10.2993,  -2.6751, -18.3701, -19.2163,   2.0688, -16.6968,
          6.9381,   9.2891,   9.6253, -17.6375,   8.7603, -19.6694,   3.6006,
        -15.9777,  -9.7883, -23.4793,  11.1518,  11.1809, -17.9926, -29.8217,
          3.4903, -27.7171, -17.2333,  -0.8779,  35.4787,  -7.9751,   7.3893,
         -6.0221,   5.6292, -26.7832,  22.5092, -13.0933, -24.3885, -12.3344,
         -5.0058,  -9.6279, -10.0285,   7.1567, -15.8241,  -4.1815,  -7.9622,
         -9.6781, -10.4838,  15.3938, -27.1653, -22.0371,  27.1375,  14.2642,
         -9.8188,  18.0297, -24.0068, -16.3649,  -3.0603,   9.8747, -19.9288,
          3.6938, -14.0305,  55.7360, -11.2336, -10.0156,  35.8923,  18.0229,
         50.7309,   1.1585,  -3.0536,  -2.9490,  25.1285, -21.8195,  20.6001,
         -4.2272,  -8.5257,  18.2717, -19.0190,  22.2985,  28.7954, -14.1726,
         15.7265,  10.4308,  35.0188, -26.4644,  12.5431,   0.58

In [21]:
VG = []

for i in range(num_actions):    
    pair_1 = []
    pair_2 = []
    
    #iterate through all entities in pair 1 of this action step
    #computing score with 
    for ent_1 in E[i][0]:
        #100 large score (for each bbox with this entity)
        alignment_scores = (ent_1 * V[i][0:100]).sum(dim=-1)
        pair_1.append(int(alignment_scores.argmax()))
                
    for ent_2 in E[i][1]:
        alignment_scores = (ent_2 * V[i][100:]).sum(dim=-1)
        pair_2.append(int(alignment_scores.argmax()))
        
    VG.append([pair_1, pair_2])

In [114]:
VG

[[[14, 29], [78, 83]],
 [[64, 93], [98, 73]],
 [[43], [20, 20]],
 [[11], [1, 1]],
 [[98, 73], [36, 29]],
 [[69, 81], [16]],
 [[96, 52], [98, 73]],
 [[70, 78], [11]]]

In [113]:
#seems to be num_actions x num_pairs (2) x num_entities in pair x 768
len(VG_V[0][0][0])

768

In [22]:
VG_V = []

for i, pair in enumerate(VG):
    VG_V.append([])
    for j, action in enumerate(pair):
        VG_V[i].append([])
        for k, entity in enumerate(action):
            if j == 0:
                VG_V[i][j].append(V[i][0:100][VG[i][j][k]])
            else:
                VG_V[i][j].append(V[i][100:][VG[i][j][k]])

In [23]:
VG

[[[14, 29], [78, 83]],
 [[64, 93], [98, 73]],
 [[43], [20, 20]],
 [[11], [1, 1]],
 [[98, 73], [36, 29]],
 [[69, 81], [16]],
 [[96, 52], [98, 73]],
 [[70, 78], [11]]]

# Training

In [1]:
def get_loss(E, V):
    loss = 0

    for entity, box in zip(E, V):
        loss = loss + loss_pair(entity, box)
        
    print(int(loss))
        
    return loss

#E and V are of shape 2 x num-entities x 768
def loss_pair(E, V):
    delta = torch.full((1, 1), 1000.0).to(device)
    
    loss = 0

    #assert(len(V) == len(E))

    num_actions = len(E)
    
    zero = torch.zeros((1)).to(device)

    for l in range(num_actions):
        for m in range(num_actions):
            before_delta = S_lm(l, m, E, V) - S_lm(l, l, E, V)
            loss = loss + torch.max(zero, (before_delta + delta))[0]

        for m in range(num_actions):
            before_delta = S_lm(m, l, E, V) - S_lm(l, l, E, V)
            loss = loss + torch.max(zero, (before_delta + delta))[0]
            
    #print("Loss: {}".format(loss))
            
    return loss

#INPUTS are ENTITIES and V for each action step
def S_lm(l, m, E, V):
    #print("Computing S_{}{}".format(l, m))
    
    entities = E[m]
    scores = 0
    
    #boxes_l_pair = torch.stack([V[:100, :], V[100:,:]])
    
    #pick pair 1 or 0 from l specified
    if l == 0:
        boxes_l = V[:100, :]
    else:
        boxes_l = V[100:, :]
        
    #if (len(VG_V[l]) == 0):
    #    return 0
    
    #boxes = torch.stack(VG_V[l])
    #print(boxes.size())
    for j, ent in enumerate(entities):
        #boxes_l should be 100 x 768
            #print(boxes_l.size())
            #broadcasting to get dot prouct with all bboxes in step l (pair part)
            #l = 0 or 1
            max_score = (ent*boxes_l).sum(dim=-1).max()
            scores += max_score
            #scores = scores + (ent * boxes_l).sum(dim=-1).max()
        
    return scores

In [2]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import loss_RA_MIL
from transformers import get_linear_schedule_with_warmup
from model import Model
from model_FC import ModelFC

import numpy as np
import torch
import matplotlib.pyplot as plt

def train(model, num_actions, batch_size, epochs=25, lr=0.001, y=0.5, MAX_DETECTIONS=20):
    dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    train_size = int(len(dataset) * (2/3))
    valid_size = int(len(dataset) - train_size)
    
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])
    
    collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(0.2*epochs), epochs)

    train_loss = np.zeros(epochs)
    valid_loss = np.zeros(epochs)
    
    model.train()
    
    batch = 8
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        num_batches = 0
        loss_ = 0
        for idx, data in enumerate(train_dataloader):
            _, bboxes_tensor, features_tensor, steps_list, entity_list, entity_count_list, _, _ = data
            batch_size = len(data[0])
            
            # Zero out any gradients.
            optimizer.zero_grad()
            
            # Run inference (forward pass).
            E, V, outputs, inputs = model(batch_size, num_actions + 1, steps_list, features_tensor, bboxes_tensor, entity_count_list, entity_list)            
            
            # Loss from alignment.
            loss_ = loss_ + get_loss(E, V)

            # Backpropagation (backward pass).
            if idx % batch == 0:
                loss_.backward()

            # Update parameters.
            optimizer.step()
            
            epoch_loss += loss_
            num_batches += 1
        
        # learning rate schedule
        # update after each epoch
        scheduler.step()
        epoch_loss = epoch_loss / num_batches
        
        # Save loss and accuracy at each epoch, plot (and checkpoint).
        train_loss[epoch] = epoch_loss
        valid_loss[epoch] = get_validation_loss(model, num_actions, y, valid_dataloader)
        
        # after epoch completes
        print("Epoch {} - Train Loss: {}, Validation Loss: {}".format(epoch + 1, train_loss[epoch], valid_loss[epoch]))
    
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.legend()
    plt.show()
        
    return train_loss, valid_loss

def get_validation_loss(model, num_actions, y, valid_dataloader):
    epoch_loss = 0.0
    num_batches = 0
        
    with torch.no_grad():
        for data in valid_dataloader:
            _, bboxes_tensor, features_tensor, steps_list, entity_list, entity_count_list, _, _ = data
            batch_size = len(data[0])

            # Run inference (forward pass).
            E, V = model(batch_size, num_actions + 1, steps_list, features_tensor, bboxes_tensor, entity_count_list, entity_list)

            # Loss from alignment.
            loss_ = get_loss(E, V)
            
            epoch_loss += loss_
            num_batches += 1
            
    epoch_loss = epoch_loss / num_batches
    
    return epoch_loss

PyTorch version 1.6.0 available.


In [3]:
import itertools
import torch
import einops
import torch.nn.functional as F

from transformers import LxmertModel, LxmertTokenizer
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from loss import loss_RA_MIL

from model import *

from random import choice
from random import seed

class ModelTesting(nn.Module):
    ACTION = '[unused3]'

    DETECTION_EMBEDDING_SIZE = 2048
    OUTPUT_EMBEDDING_SIZE = 768

    def __init__(self, device, NUM_FRAMES_PER_STEP=5, MAX_DETECTIONS=20):
        super(ModelTesting, self).__init__()

        self.device = device

        self.NUM_FRAMES_PER_STEP = NUM_FRAMES_PER_STEP
        self.MAX_DETECTIONS = MAX_DETECTIONS
        self.CANDIDATES = self.NUM_FRAMES_PER_STEP * self.MAX_DETECTIONS

        self.lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.lxmert_tokenizer.add_special_tokens({"additional_special_tokens": [self.ACTION]})
        self.lxmert_tokenizer.encode([self.ACTION])

        self.ACTION_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(self.ACTION)

        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.lxmert.to(device)

    def forward(self, BATCH_SIZE, NUM_ACTIONS, steps_list, features, boxes, entity_count_list, entity_list):
        assert(BATCH_SIZE == 1)
        
        NUM_ACTIONS = NUM_ACTIONS - 1
        
        steps_list = remove_unused2(steps_list)
        
        entities_count = entity_count_list[0]
        entities = entity_list[0]
        steps = steps_list[0]
        steps = [step.strip() for step in steps.split(self.ACTION)[:-2]]
        
        boxes = boxes.squeeze(0)
        boxes = boxes.to(self.device)
        boxes = boxes.reshape(NUM_ACTIONS, self.CANDIDATES, -1)
        
        features = features.squeeze(0)
        features = features.to(self.device)
        features = features.reshape(NUM_ACTIONS, self.CANDIDATES, -1)

        steps_pairs = []
        
        bboxes_pairs = torch.zeros(NUM_ACTIONS, 2, self.CANDIDATES, 4)
        bboxes_pairs = bboxes_pairs.to(self.device)
        
        features_pairs = torch.zeros(NUM_ACTIONS, 2, self.CANDIDATES, self.DETECTION_EMBEDDING_SIZE)
        features_pairs = features_pairs.to(self.device)
        
        entity_list_pairs = []
        num_steps = len(steps)
        pairs = []
        
        for idx, step in enumerate(steps):
            idx_1 = idx
            idx_2 = choice([idx_2 for idx_2 in range(num_steps) if idx_2 != idx_1])

            pairs.append((idx_1, idx_2))

            steps_pairs.append(steps[idx_1] + " " + self.ACTION + " " + steps[idx_2] + " " + self.ACTION + " " + self.ACTION)

            bboxes_pairs[idx_1][0] = boxes[idx_1]
            bboxes_pairs[idx_1][1] = boxes[idx_2]

            features_pairs[idx_1][0] = features[idx_1]
            features_pairs[idx_1][1] = features[idx_2]

            entity_list_pairs.append([entities[idx_1], entities[idx_2]])
        #print(entity_list_pairs)    
        
        bboxes_pairs = bboxes_pairs.reshape(NUM_ACTIONS, 2 * self.CANDIDATES, -1)
        bboxes_pairs = bboxes_pairs.to(self.device)

        features_pairs = features_pairs.reshape(NUM_ACTIONS, 2 * self.CANDIDATES, self.DETECTION_EMBEDDING_SIZE)
        features_pairs = features_pairs.to(self.device)

        entity_count = [len(action) for action in sum(entity_list_pairs, [])]
        
        inputs = self.lxmert_tokenizer(
            steps_pairs,
            padding="longest",
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        inputs.input_ids = inputs.input_ids.to(self.device)
        inputs.attention_mask = inputs.attention_mask.to(self.device)
        inputs.token_type_ids = inputs.token_type_ids.to(self.device)

        output = self.lxmert(
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    visual_feats=features_pairs,
                    visual_pos=bboxes_pairs,
                    token_type_ids=inputs.token_type_ids,
                    return_dict=True,
                    output_attentions=True
                )
        
        entity_idx = get_ent_inds(self, entity_list_pairs, steps_pairs)
        entity_embeddings = get_entity_embeddings(output['language_output'], entity_idx).split(entity_count)

        action_idx = (inputs.input_ids == self.ACTION_TOKEN)
        A = output['language_output'][action_idx]

        V = output['vision_output']
        
        E = []
        for i in range(NUM_ACTIONS):
            idx_1 = 2 * i
            idx_2 = idx_1 + 1
            E.append([entity_embeddings[idx_1], entity_embeddings[idx_2]])
        
        VG = []
        
        for i in range(NUM_ACTIONS):    
            pair_1 = []
            pair_2 = []

            for ent_1 in E[i][0]:
                alignment_scores = (ent_1 * V[i][0:100]).sum(dim=-1)
                pair_1.append(alignment_scores.argmax())

            for ent_2 in E[i][1]:
                alignment_scores = (ent_2 * V[i][100:]).sum(dim=-1)
                pair_2.append(alignment_scores.argmax())

            VG.append([pair_1, pair_2])
            
        VG_V = []

        for i, pair in enumerate(VG):
            VG_V.append([])
            for j, action in enumerate(pair):
                VG_V[i].append([])
                for k, entity in enumerate(action):
                    if j == 0:
                        VG_V[i][j].append(V[i][0:100][VG[i][j][k]])
                    else:
                        VG_V[i][j].append(V[i][100:][VG[i][j][k]])
                        
        VG_ind = torch.zeros((1, NUM_ACTIONS, max(entity_count))).to(self.device)

        for i, pair in enumerate(VG):
            for k, entity in enumerate(pair[0]):
                VG_ind[0][i][k] = VG[i][0][k]
                        
        #return None, None, None, NUM_ACTIONS, None, None, VG_ind, None
        return E, V, output, inputs

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_test = ModelTesting(device)

In [5]:
dataset = YouCookII(8, "/h/sagar/ece496-capstone/datasets/ycii")
collate = YouCookIICollate(MAX_DETECTIONS=20)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate)

In [7]:
_, boxes, features, steps_list, entity_list, entity_count_list, _, _ = next(iter(dataloader))

In [8]:
E, V, outputs, inputs = model_test(1, 8 + 1, steps_list, features, boxes, entity_count_list, entity_list)

In [27]:
E, V, outputs, inputs = model_test(1, 8 + 1, steps_list, features, boxes, entity_count_list, entity_list)

[[['garram masala seeds', 'the oil'], ['water', 'the pot']], [['the lamb', 'the pot'], ['chili', 'the pot']], [['garlic ginger paste'], ['chili', 'the pot']], [['chili tumeric coriander cumin'], ['garlic ginger paste']], [['water', 'the pot'], ['chili tumeric coriander cumin']], [['potatos', 'the pot'], ['garlic ginger paste']], [['the tomatos', 'the pot'], ['water', 'the pot']], [['chili', 'the pot'], ['the lamb', 'the pot']]]


In [9]:
tokens = model_test.lxmert_tokenizer.convert_ids_to_tokens(inputs.input_ids[0])

In [10]:
tokens_1 = tokens
tokens_2 = ["C{}".format(i + 1) for i in range(200)]

In [11]:
action = 6

attention_1 = [attn[action].unsqueeze(0) for attn in outputs.language_attentions]
attention_2 = [attn[action].unsqueeze(0) for attn in outputs.vision_attentions]

cross = [attn[action].unsqueeze(0) for attn in outputs.cross_encoder_attentions]

In [12]:
cross

[tensor([[[[0.0003, 0.0022, 0.0010,  ..., 0.0013, 0.0001, 0.0048],
           [0.0031, 0.0017, 0.0021,  ..., 0.0043, 0.0035, 0.0049],
           [0.0042, 0.0021, 0.0047,  ..., 0.0030, 0.0097, 0.0071],
           ...,
           [0.0085, 0.0020, 0.0031,  ..., 0.0049, 0.0074, 0.0026],
           [0.0081, 0.0022, 0.0029,  ..., 0.0046, 0.0069, 0.0027],
           [0.0068, 0.0025, 0.0059,  ..., 0.0021, 0.0060, 0.0096]],
 
          [[0.0011, 0.0016, 0.0011,  ..., 0.0040, 0.0118, 0.0058],
           [0.0025, 0.0054, 0.0030,  ..., 0.0046, 0.0022, 0.0066],
           [0.0032, 0.0078, 0.0029,  ..., 0.0071, 0.0041, 0.0073],
           ...,
           [0.0079, 0.0090, 0.0079,  ..., 0.0057, 0.0026, 0.0021],
           [0.0074, 0.0073, 0.0069,  ..., 0.0053, 0.0024, 0.0020],
           [0.0040, 0.0069, 0.0039,  ..., 0.0013, 0.0332, 0.0183]],
 
          [[0.0157, 0.0104, 0.0076,  ..., 0.0070, 0.0595, 0.0036],
           [0.0037, 0.0040, 0.0040,  ..., 0.0034, 0.0224, 0.0145],
           [0.0027, 0.00

In [161]:
cross

[tensor([[[[1.1291e-03, 9.5218e-03, 1.1222e-02,  ..., 4.7141e-04,
            7.4418e-03, 4.3870e-04],
           [3.7549e-03, 2.1468e-03, 2.5470e-03,  ..., 5.3647e-03,
            4.3351e-03, 5.9800e-03],
           [3.1675e-03, 1.7758e-03, 2.4763e-03,  ..., 4.6627e-03,
            6.9895e-03, 6.5875e-03],
           ...,
           [4.5634e-03, 1.8112e-03, 1.3702e-03,  ..., 2.8315e-03,
            3.6888e-03, 6.5386e-03],
           [4.5718e-03, 2.2012e-03, 1.5567e-03,  ..., 3.1205e-03,
            4.4063e-03, 5.9100e-03],
           [3.8289e-03, 2.0482e-03, 1.4857e-03,  ..., 2.1926e-03,
            7.5698e-03, 8.3844e-03]],
 
          [[2.0023e-03, 5.5029e-03, 1.3176e-03,  ..., 1.9449e-03,
            9.2574e-03, 1.5056e-03],
           [2.6108e-03, 5.0265e-03, 2.9578e-03,  ..., 4.1859e-03,
            1.0461e-02, 5.2048e-03],
           [3.2827e-03, 1.0099e-02, 3.5019e-03,  ..., 4.9771e-03,
            3.7235e-03, 5.9425e-03],
           ...,
           [4.0844e-03, 3.8357e-03, 4.

In [159]:
torch.stack(cross).min()

tensor(3.8600e-08, device='cuda:0', grad_fn=<MinBackward1>)

In [160]:
torch.stack(cross).max()

tensor(0.2424, device='cuda:0', grad_fn=<MaxBackward1>)

In [88]:
from bertviz import model_view
from bertviz import head_view

head_view(
    encoder_attention=attention_2,
    decoder_attention=attention_1,
    cross_attention=cross,
    encoder_tokens=tokens_2,
    decoder_tokens=tokens_1,
    layer=0,
    heads=[2]
)

ModuleNotFoundError: No module named 'bertviz'

In [6]:
train_loss, valid_loss = train(model_test, 8, 1, epochs=100, lr=1e-2)

64007
63993
63987
63970
64084
63999


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.76 GiB total capacity; 13.24 GiB already allocated; 13.44 MiB free; 13.85 GiB reserved in total by PyTorch)

In [120]:
from eval_fi import eval_all_dataset
eval_all_dataset(model_test)

--------------------------------------------------
Action 1: in a bowl, add ground beef, salt, pepper, 1 egg, dijon mustard and bread crumbs
Best IoU possible = 0.814872145652771
Random Candidate IoU = 0.36690545082092285
Chosen Frame IoU: 0.26270750164985657
Best IoU possible = 0.5932643413543701
Random Candidate IoU = 0.3814815580844879
Chosen Frame IoU: 0.37369081377983093
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: mix the ingredients with hand
Best IoU possible = 0.6150804162025452
Random Candidate IoU = 0.34453853964805603
Chosen Frame IoU: 0.24410486221313477
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add vegetable oil in a pan
Best IoU possible = 0.7381150722503662
Random Candid

Best IoU possible = 0.6265260577201843
Random Candidate IoU = 0.4184003174304962
Chosen Frame IoU: 0.34778180718421936
Mean Upper Bound IoU: 0.6996067762374878, Mean Random IoU: 0.3719481825828552, Mean Model IoU: 0.20323562622070312
Top-1 acc@0.5:
Proposal Upper Bound: 0.9375, Random: 0.3125, Model: 0.0
--------------------------------------------------
Action 1: cut the cabbage into small pieces
Best IoU possible = 0.8641247749328613
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.05055755004286766
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: Add carawat seeds, sea salt
Best IoU possible = 0.11877390742301941
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: mix them evenly in the cabbage
Best IoU possible = 0.22755128145217896
Random Candidate IoU = 0.06622441112995148
Chosen Frame IoU: 0.0
Best IoU possible = 0.82718759

--------------------------------------------------
Action 1: Soak the ribs in water
Best IoU possible = 0.5332905414373129
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.061612147837877274
Best IoU possible = 0.632248044013977
Random Candidate IoU = 0.03894210606813431
Chosen Frame IoU: 0.19863736629486084
--------------------------------------------------
Action 2: peel the apple and kiwi
Best IoU possible = 0.6564290527706349
Random Candidate IoU = 0.028097480535507202
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: cut the apple and kiwi
Best IoU possible = 0.1569916644163539
Random Candidate IoU = 0.04559415497345186
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: cut the onion and garlic
Best IoU possible = 0.03550495477541392
Random Candidate IoU = 0.023895193921405905
Chosen Frame IoU: 0.0
Best IoU possible = 0.5952157974

Best IoU possible = 0.6936541199684143
Random Candidate IoU = 0.22794680297374725
Chosen Frame IoU: 0.35625985264778137
--------------------------------------------------
Action 7: Coat the fish strips with flour and the mixture
Best IoU possible = 0.6177937388420105
Random Candidate IoU = 0.5476809144020081
Chosen Frame IoU: 0.13239791989326477
Best IoU possible = 0.616290271282196
Random Candidate IoU = 0.06289678812026978
Chosen Frame IoU: 0.4822232127189636
This entity has no ground truth bounding box
--------------------------------------------------
Action 8: fry them in the oil
Best IoU possible = 0.8227578401565552
Random Candidate IoU = 0.2759611904621124
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 9: Drain the fried fish
Best IoU possible = 0.6309415102005005
Random Candidate IoU = 0.36201441287994385
Chosen Frame IoU: 0.07456156611442566
--------------------------------------------------
Action 

Best IoU possible = 0.6099690794944763
Random Candidate IoU = 0.06688570231199265
Chosen Frame IoU: 0.0
Best IoU possible = 0.5592836737632751
Random Candidate IoU = 0.024766597896814346
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 6: beat some eggs
Best IoU possible = 0.70316082239151
Random Candidate IoU = 0.07153047621250153
Chosen Frame IoU: 0.14509880542755127
--------------------------------------------------
Action 7: whisk them well
Best IoU possible = 0.702238917350769
Random Candidate IoU = 0.0008182955207303166
Chosen Frame IoU: 0.12606896460056305
--------------------------------------------------
Action 8: Add some water, ketchup and garlic powder while whisking
Best IoU possible = 0.5713340640068054
Random Candidate IoU = 0.22607269883155823
Chosen Frame IoU: 0.4243323802947998
Best IoU possible = 0.4972221255302429
Random Candidate IoU = 0.34611961245536804
Chosen Frame IoU: 0.2949502468109131
This entity has no ground truth bounding bo

...

Chosen Frame IoU: 0.08123277872800827
--------------------------------------------------
Action 10: wrap the cabbage
Best IoU possible = 0.47764015197753906
Random Candidate IoU = 0.085892971592562
Chosen Frame IoU: 0.12364049243689879
--------------------------------------------------
Action 11: drain the water out of it
This entity has no ground truth bounding box
Best IoU possible = 0.04125465495055638
Random Candidate IoU = 0.0027251674327999353
Chosen Frame IoU: 0.04125465495055638
--------------------------------------------------
Action 12: chop it using a meat shopper
Best IoU possible = 0.0639820263431321
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.004741316195577383
This entity has no ground truth bounding box
--------------------------------------------------
Action 13: Add half a stick of butter
Best IoU possible = 0.42732922247159943
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 14: and mix it well
Best IoU po

Best IoU possible = 0.6838409304618835
Random Candidate IoU = 0.351907879114151
Chosen Frame IoU: 0.25424841046333313
--------------------------------------------------
Action 22: Season avocado slices with little bit of salt
Best IoU possible = 0.6126750707626343
Random Candidate IoU = 0.06889123469591141
Chosen Frame IoU: 0.18262402713298798
This entity has no ground truth bounding box
--------------------------------------------------
Action 23: top the avocadoes with lettuce leaves
Best IoU possible = 0.1775994747877121
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.10021654516458511
Best IoU possible = 0.15419260964273693
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.039154596626758575
--------------------------------------------------
Action 24: Place the avocado and lettuce topped bread slice on the bacon and tomato topped bread slice
Best IoU possible = 0.7627198100090027
Random Candidate IoU = 0.2895210087299347
Chosen Frame IoU: 0.04035026580095291
Mean Upper Bound IoU: 0.55

Best IoU possible = 0.06551862602997006
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.7772270441055298
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0734008053657334
Mean Upper Bound IoU: 0.40954235196113586, Mean Random IoU: 0.004520539194345474, Mean Model IoU: 0.03360904008150101
Top-1 acc@0.5:
Proposal Upper Bound: 0.5, Random: 0.0, Model: 0.0
--------------------------------------------------
Action 1: heat two pieces of bread on the grill
Best IoU possible = 0.6943070888519287
Random Candidate IoU = 0.27869951725006104
Chosen Frame IoU: 0.15341348946094513
Best IoU possible = 0.5016346573829651
Random Candidate IoU = 0.30160313844680786
Chosen Frame IoU: 0.2469794750213623
--------------------------------------------------
Action 2: beat 6 eggs in a large cup
Best IoU possible = 0.2712264657020569
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.28724539279937744
Random Candidate IoU = 0.10667718929542494
Chosen Frame IoU: 0.0
-----

Best IoU possible = 0.27110535961674964
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.4273582023515265
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 6: mix them together
Best IoU possible = 0.5742625844098638
Random Candidate IoU = 0.07167377104924626
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 7: Cook the flavored beef on a grill
Best IoU possible = 0.28265406411409216
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.024415134181520838
Best IoU possible = 0.6871748566627502
Random Candidate IoU = 0.2313026487827301
Chosen Frame IoU: 0.0
Mean Upper Bound IoU: 0.5074010491371155, Mean Random IoU: 0.08436855673789978, Mean Model IoU: 0.013590242713689804
Top-1 acc@0.5:
Proposal Upper Bound: 0.5, Random: 0.08333333333333333, Model: 0.0
--------------------------------------------------
Action 1: mix the eggs with salt and pepper
Best IoU possible = 0.5809203443057018
Ran

--------------------------------------------------
Action 1: Heat hopped carrot, celery, garlic, onion in a large pot with oil
Best IoU possible = 0.8464573621749878
Random Candidate IoU = 0.409839928150177
Chosen Frame IoU: 0.15206922590732574
Best IoU possible = 0.5040996670722961
Random Candidate IoU = 0.283023864030838
Chosen Frame IoU: 0.27765077352523804
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: Add herbs, spices, salt and pepper
Best IoU possible = 0.25859370260268155
Random Candidate IoU = 0.1583319455385208
Chosen Frame IoU: 0.05983463674783707
Best IoU possible = 0.6441613435745239
Random Candidate IoU = 0.02240215614438057
Chosen Frame IoU: 0.028988098725676537
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add tomato pas

In [116]:
#torch.save(model_test.state_dict(), "/h/sagar/ece496-capstone/weights/weights-nv-5")

## After Training 

In [32]:
model_test.load_state_dict(torch.load("/h/sagar/ece496-capstone/weights/weights-nv-5"))

<All keys matched successfully>

In [33]:
len(inputs.input_ids)

8

In [34]:
tokens = model_test.lxmert_tokenizer.convert_ids_to_tokens(inputs.input_ids[1])

In [35]:
entity_list

[[['garram masala seeds', 'the oil'],
  ['the lamb', 'the pot'],
  ['garlic ginger paste'],
  ['chili tumeric coriander cumin'],
  ['water', 'the pot'],
  ['potatos', 'the pot'],
  ['the tomatos', 'the pot'],
  ['chili', 'the pot']]]

In [172]:
entity_list

[[['garram masala seeds', 'the oil'],
  ['the lamb', 'the pot'],
  ['garlic ginger paste'],
  ['chili tumeric coriander cumin'],
  ['water', 'the pot'],
  ['potatos', 'the pot'],
  ['the tomatos', 'the pot'],
  ['chili', 'the pot']]]

In [171]:
tokens

['[CLS]',
 'add',
 'the',
 'lamb',
 'to',
 'the',
 'pot',
 '.',
 '[unused3]',
 'add',
 'garlic',
 'ginger',
 'paste',
 'and',
 'chopped',
 'onions',
 'to',
 'the',
 'pot',
 '.',
 '[unused3]',
 '[unused3]',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [36]:
E, VG_V, outputs, inputs = model_test(1, 8 + 1, steps_list, features, boxes, entity_count_list, entity_list)


tokens = model_test.lxmert_tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
tokens_1 = tokens
tokens_2 = ["C{}".format(i + 1) for i in range(200)]
action = 6

attention_1 = [attn[action].unsqueeze(0) for attn in outputs.language_attentions]
attention_2 = [attn[action].unsqueeze(0) for attn in outputs.vision_attentions]

cross = [attn[action].unsqueeze(0) for attn in outputs.cross_encoder_attentions]

[[['garram masala seeds', 'the oil'], ['potatos', 'the pot']], [['the lamb', 'the pot'], ['garlic ginger paste']], [['garlic ginger paste'], ['chili tumeric coriander cumin']], [['chili tumeric coriander cumin'], ['the lamb', 'the pot']], [['water', 'the pot'], ['chili', 'the pot']], [['potatos', 'the pot'], ['garram masala seeds', 'the oil']], [['the tomatos', 'the pot'], ['water', 'the pot']], [['chili', 'the pot'], ['the tomatos', 'the pot']]]


In [38]:
cross

[tensor([[[[0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           ...,
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050]],
 
          [[0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           ...,
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050]],
 
          [[0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.0050, 0.0050,  ..., 0.0050, 0.0050, 0.0050],
           [0.0050, 0.00

In [46]:
outputs['vision_output']

tensor([[[-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         ...,
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352]],

        [[-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         ...,
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352]],

        [[-1.2660, -1.2161, -1.2227,  ...,  1.0998, -1.2849, -1.2352],
         [-1.2660, -1.2161, -1.2227,  ...,  1

In [47]:
get_loss(E, VG_V)

74058


tensor([74058.9531], device='cuda:0', grad_fn=<AddBackward0>)

In [48]:
VG_V

[[[tensor([-1.2660e+00, -1.2161e+00, -1.2227e+00, -4.4666e-01, -1.7893e+00,
            1.5246e+00, -1.3926e+00,  1.6306e+00,  1.4970e+00,  6.3761e-01,
           -1.1344e+00,  1.4219e+00, -1.2732e+00,  1.7557e+00,  8.5928e-01,
            1.8234e+00,  6.7149e-01,  1.7644e+00, -1.7794e+00, -1.8247e+00,
            1.3498e+00, -1.4064e+00,  3.4099e-03,  1.6860e+00,  8.6817e-01,
           -1.7724e+00, -1.0886e+00, -1.4576e+00,  1.8946e+00,  1.8360e+00,
            1.2359e+00,  1.6859e+00,  1.6316e+00,  5.3099e-01,  1.0267e+00,
            7.7830e-01,  1.4985e+00, -1.7334e+00,  3.3154e+00, -1.3666e+00,
            1.9439e+00,  1.7329e+00,  1.3116e+00,  1.9908e+00, -9.4174e-01,
            1.7918e+00, -1.7680e+00, -1.7290e+00, -1.6709e+00, -1.8215e+00,
            1.7171e+00, -1.9250e+00, -1.7329e+00, -9.7956e-01,  1.7094e+00,
           -6.0189e-01,  1.1214e+00, -1.3706e+00,  1.0021e+00, -1.3865e+00,
           -1.3399e+00, -1.7834e+00, -1.4719e+00,  3.9816e-01,  1.7285e+00,
            

In [52]:
len(E)

8

In [54]:
len(E[0])

2

In [41]:
entity_idx = get_ent_inds(model, entity_list_pairs, steps_pairs)
entity_embeddings = get_entity_embeddings(outputs['language_output'], entity_idx).split(entity_count)

action_idx = (inputs.input_ids == ACTION_TOKEN)
A = outputs['language_output'][action_idx]

V = outputs['vision_output']

In [43]:
V.size()

torch.Size([8, 200, 768])

In [123]:
VG_V #PROBABLY THE BBOX EMBEDDINGS FOR "MOST ALIGNED" BBOXES

[[[tensor([-1.2660e+00, -1.2161e+00, -1.2227e+00, -4.4666e-01, -1.7893e+00,
            1.5246e+00, -1.3926e+00,  1.6306e+00,  1.4970e+00,  6.3761e-01,
           -1.1344e+00,  1.4219e+00, -1.2732e+00,  1.7557e+00,  8.5928e-01,
            1.8234e+00,  6.7149e-01,  1.7644e+00, -1.7794e+00, -1.8247e+00,
            1.3498e+00, -1.4064e+00,  3.4099e-03,  1.6860e+00,  8.6817e-01,
           -1.7724e+00, -1.0886e+00, -1.4576e+00,  1.8946e+00,  1.8360e+00,
            1.2359e+00,  1.6859e+00,  1.6316e+00,  5.3099e-01,  1.0267e+00,
            7.7830e-01,  1.4985e+00, -1.7334e+00,  3.3154e+00, -1.3666e+00,
            1.9439e+00,  1.7329e+00,  1.3116e+00,  1.9908e+00, -9.4174e-01,
            1.7918e+00, -1.7680e+00, -1.7290e+00, -1.6709e+00, -1.8215e+00,
            1.7171e+00, -1.9250e+00, -1.7329e+00, -9.7956e-01,  1.7094e+00,
           -6.0189e-01,  1.1214e+00, -1.3706e+00,  1.0021e+00, -1.3865e+00,
           -1.3399e+00, -1.7834e+00, -1.4719e+00,  3.9816e-01,  1.7285e+00,
            

In [125]:
loss_pair(E[0], VG_V[0])

tensor([8000.], device='cuda:0', grad_fn=<AddBackward0>)

In [126]:
get_loss(E, VG_V)

74058


tensor([74058.9531], device='cuda:0', grad_fn=<AddBackward0>)

In [129]:
count=0
for entity, box in zip(E, VG_V):
    print(count)
    print(loss_pair(entity,box))
    count+=1

0
tensor([8000.], device='cuda:0', grad_fn=<AddBackward0>)
1
tensor([13029.4814], device='cuda:0', grad_fn=<AddBackward0>)
2
tensor([7999.9990], device='cuda:0', grad_fn=<AddBackward0>)
3
tensor([13029.4639], device='cuda:0', grad_fn=<AddBackward0>)
4
tensor([7999.9980], device='cuda:0', grad_fn=<AddBackward0>)
5
tensor([8000.], device='cuda:0', grad_fn=<AddBackward0>)
6
tensor([8000.0039], device='cuda:0', grad_fn=<AddBackward0>)
7
tensor([7999.9980], device='cuda:0', grad_fn=<AddBackward0>)


In [176]:
a1_ents = E[1]
#should be of shape 2 x num_ents x 768
#here (in action step 1) the entity list_pairs is ['the lamb', 'the pot'], ['garlic ginger paste'] (so 2 ents, then 1)

In [177]:
a1_ents[1]

tensor([[-1.2254e+00, -1.4670e+00, -1.1311e+00, -4.5796e-01, -1.7300e+00,
          1.6171e+00, -4.8832e-01,  1.9444e+00,  1.2740e+00,  5.1137e-01,
         -8.7808e-01,  1.5453e+00, -1.4919e+00,  1.6166e+00,  3.3171e+00,
          1.7833e+00,  5.7950e-01,  1.4433e+00, -1.8348e+00, -1.3928e+00,
          9.3113e-01, -1.2909e+00, -1.1333e-03,  1.6608e+00,  8.3705e-01,
         -1.9114e+00, -1.9410e+00, -1.7474e+00,  1.7448e+00,  1.8003e+00,
          1.7077e+00,  1.1470e+00,  1.5339e+00,  1.5284e+00,  1.4016e+00,
          7.1970e-01,  1.4518e+00, -1.5734e+00, -3.2905e+01, -1.9024e+00,
          2.0125e+00,  1.6260e+00,  1.4463e+00,  2.0916e+00, -1.3323e+00,
          2.3155e+00, -1.7465e+00, -1.5136e+00, -1.5964e+00, -1.8374e+00,
          1.5593e+00, -1.6877e+00, -1.7380e+00, -1.2832e+00,  1.3601e+00,
         -3.4260e+00,  1.3872e+00, -1.2850e+00,  8.6544e-01, -1.7548e+00,
         -1.0284e+00, -1.7077e+00, -1.4622e+00,  3.9461e-01,  1.7950e+00,
          1.0852e+00,  1.7874e+00,  1.

In [178]:
loss_pair(E[1], VG_V[1])

tensor([13029.4814], device='cuda:0', grad_fn=<AddBackward0>)

In [192]:
S_lm(1,0, E[1], VG_V[1])

tensor(-12058.9561, device='cuda:0', grad_fn=<AddBackward0>)

In [184]:
E[1]

[tensor([[-1.2254, -1.4670, -1.1311,  ...,  1.3345, -0.8106, -3.0416],
         [-1.2254, -1.4670, -1.1311,  ...,  1.3345, -0.8106, -3.0416]],
        device='cuda:0', grad_fn=<SplitWithSizesBackward>),
 tensor([[-1.2254e+00, -1.4670e+00, -1.1311e+00, -4.5796e-01, -1.7300e+00,
           1.6171e+00, -4.8832e-01,  1.9444e+00,  1.2740e+00,  5.1137e-01,
          -8.7808e-01,  1.5453e+00, -1.4919e+00,  1.6166e+00,  3.3171e+00,
           1.7833e+00,  5.7950e-01,  1.4433e+00, -1.8348e+00, -1.3928e+00,
           9.3113e-01, -1.2909e+00, -1.1333e-03,  1.6608e+00,  8.3705e-01,
          -1.9114e+00, -1.9410e+00, -1.7474e+00,  1.7448e+00,  1.8003e+00,
           1.7077e+00,  1.1470e+00,  1.5339e+00,  1.5284e+00,  1.4016e+00,
           7.1970e-01,  1.4518e+00, -1.5734e+00, -3.2905e+01, -1.9024e+00,
           2.0125e+00,  1.6260e+00,  1.4463e+00,  2.0916e+00, -1.3323e+00,
           2.3155e+00, -1.7465e+00, -1.5136e+00, -1.5964e+00, -1.8374e+00,
           1.5593e+00, -1.6877e+00, -1.7380e+00

In [185]:
VG_V[1]

[[tensor([-1.2660e+00, -1.2161e+00, -1.2227e+00, -4.4666e-01, -1.7893e+00,
           1.5246e+00, -1.3926e+00,  1.6306e+00,  1.4970e+00,  6.3761e-01,
          -1.1344e+00,  1.4219e+00, -1.2732e+00,  1.7557e+00,  8.5928e-01,
           1.8234e+00,  6.7149e-01,  1.7644e+00, -1.7794e+00, -1.8247e+00,
           1.3498e+00, -1.4064e+00,  3.4099e-03,  1.6860e+00,  8.6817e-01,
          -1.7724e+00, -1.0886e+00, -1.4576e+00,  1.8946e+00,  1.8360e+00,
           1.2359e+00,  1.6859e+00,  1.6316e+00,  5.3099e-01,  1.0267e+00,
           7.7830e-01,  1.4985e+00, -1.7334e+00,  3.3154e+00, -1.3666e+00,
           1.9439e+00,  1.7329e+00,  1.3116e+00,  1.9908e+00, -9.4174e-01,
           1.7918e+00, -1.7680e+00, -1.7290e+00, -1.6709e+00, -1.8215e+00,
           1.7171e+00, -1.9250e+00, -1.7329e+00, -9.7956e-01,  1.7094e+00,
          -6.0189e-01,  1.1214e+00, -1.3706e+00,  1.0021e+00, -1.3865e+00,
          -1.3399e+00, -1.7834e+00, -1.4719e+00,  3.9816e-01,  1.7285e+00,
           7.8351e-01,  1

In [134]:
len(E[1])

2

In [135]:
len(E[0])

2

In [188]:
E[1][0]

tensor([[-1.2254, -1.4670, -1.1311,  ...,  1.3345, -0.8106, -3.0416],
        [-1.2254, -1.4670, -1.1311,  ...,  1.3345, -0.8106, -3.0416]],
       device='cuda:0', grad_fn=<SplitWithSizesBackward>)

In [194]:
boxes = torch.stack(VG_V[1][0])

In [196]:
(E[1][0][0]*boxes).sum(dim=-1)

tensor([-6029.4780, -6029.4780], device='cuda:0', grad_fn=<SumBackward1>)

In [206]:
len(VG_V[1][1][0])

768

In [75]:
get_loss(E,V)

74058


tensor([74058.9453], device='cuda:0', grad_fn=<AddBackward0>)

In [69]:
i=0
for ent, b in zip(E,V):
    print(i)
    i+=1

0
1
2
3
4
5
6
7


In [76]:
def get_loss(E, V):
    loss = 0

    for entity, box in zip(E, V):
        loss = loss + loss_pair(entity, box)
        
    print(int(loss))
        
    return loss

#E and V are of shape 2 x num-entities x 768
def loss_pair(E, V):
    delta = torch.full((1, 1), 1000.0).to(device)
    
    loss = 0

    #assert(len(V) == len(E))

    num_actions = len(E)
    
    zero = torch.zeros((1)).to(device)

    for l in range(num_actions):
        for m in range(num_actions):
            before_delta = S_lm(l, m, E, V) - S_lm(l, l, E, V)
            loss = loss + torch.max(zero, (before_delta + delta))[0]

        for m in range(num_actions):
            before_delta = S_lm(m, l, E, V) - S_lm(l, l, E, V)
            loss = loss + torch.max(zero, (before_delta + delta))[0]
            
    #print("Loss: {}".format(loss))
            
    return loss

#INPUTS are ENTITIES and V for each action step
def S_lm(l, m, E, V):
    #print("Computing S_{}{}".format(l, m))
    
    entities = E[m]
    scores = 0
    
    boxes_l_pair = torch.stack([V[:100, :], V[100:,:]])
    
    #pick pair 1 or 0 from l specified
    boxes_l = boxes_l_pair[l]
    #if (len(VG_V[l]) == 0):
    #    return 0
    
    #boxes = torch.stack(VG_V[l])
    #print(boxes.size())
    for j, ent in enumerate(entities):
        #boxes_l should be 100 x 768
            #print(boxes_l.size())
            #broadcasting to get dot prouct with all bboxes in step l (pair part)
            #l = 0 or 1
            max_score = (ent*boxes_l).sum(dim=-1).max()
            scores += max_score
            #scores = scores + (ent * boxes_l).sum(dim=-1).max()
        
    return scores

In [62]:
i = 1
S_lm(1,1, E[i], V[i])

tensor(-6029.4727, device='cuda:0', grad_fn=<AddBackward0>)

In [203]:
S_lm(0,0, E[1], VG_V[1])

torch.Size([2, 768])


tensor(-12058.9551, device='cuda:0', grad_fn=<AddBackward0>)

In [45]:
boxes_l = torch.stack([V[0][:100, :], V[0][100:, :]])

In [46]:
boxes_l.size()

torch.Size([2, 100, 768])